In [ ]:
import sys
import pathlib
import numpy
import scipy.interpolate
import ipywidgets
import pandas
from h5py import File as h5open
from cycler import cycler
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import ticker
from matplotlib import cm
from matplotlib.legend_handler import HandlerTuple

In [ ]:
# find helpers and locate workdir
for parent in [pathlib.Path.cwd()] + list(pathlib.Path.cwd().parents):
    if parent.joinpath("modulus").is_dir():
        projdir = parent
        sys.path.insert(0, str(projdir.joinpath("modulus")))
        from helpers.utils import read_tensorboard_data  # pylint: disable=import-error
        from helpers.lr_simulator import cyclic_exp_range, exponential, tf_exponential
        break
else:
    raise FileNotFoundError("Couldn't find module `helpers`.")

# point workdir to the correct folder
workdir = projdir.joinpath("modulus", "cylinder-2d-re200")
petibmdir = projdir.joinpath("petibm", "cylinder-2d-re200")

In [ ]:
# unified figure style
pyplot.style.use(projdir.joinpath("resources", "figstyle"))

In [ ]:
def read_petibm_snapshot(workdir, time):
    """Read snapshots from PetIBM simulations.
    """
    assert float(int(time)) == float(time), f"Only supports integer time. Got {time}."

    # hard-coded simulation parameters; should match the config.yaml in the PetIBM case
    dt = 0.005
    step = int(time/dt+0.5)

    # get gridline and calculate coordinates
    with h5open(workdir.joinpath("output", "grid.h5"), "r") as h5file:
        coords = {
            r"$u$": numpy.meshgrid(h5file["u"]["x"][...], h5file["u"]["y"][...]),
            r"$v$": numpy.meshgrid(h5file["v"]["x"][...], h5file["v"]["y"][...]),
            r"$p$": numpy.meshgrid(h5file["p"]["x"][...], h5file["p"]["y"][...]),
            r"$\omega_z$": numpy.meshgrid(h5file["wz"]["x"][...], h5file["wz"]["y"][...]),
        }

    # get data
    with h5open(workdir.joinpath("output", f"{step:07d}.h5"), "r") as h5file:
        vals = {
            r"$u$": h5file["u"][...],
            r"$v$": h5file["v"][...],
            r"$p$": h5file["p"][...],
            r"$\omega_z$": h5file["wz"][...],
        }
    
    return coords, vals

In [ ]:
def read_pinn_snapshot(workdir, casename, time):
    """Read snapshots from PINN simulations.
    """
    with h5open(workdir.joinpath("outputs", casename).with_suffix(".h5"), "r") as h5file:
        coords = {
            r"$u$": (h5file["fields/x"][...], h5file["fields/y"][...]),
            r"$v$": (h5file["fields/x"][...], h5file["fields/y"][...]),
            r"$p$": (h5file["fields/x"][...], h5file["fields/y"][...]),
            r"$\omega_z$": (h5file["fields/x"][...], h5file["fields/y"][...]),
        }

        if not h5file.attrs["unsteady"]:
            time = "steady"

        vals = {
            r"$u$": h5file[f"fields/{time}/u"][...],
            r"$v$": h5file[f"fields/{time}/v"][...],
            r"$p$": h5file[f"fields/{time}/p"][...],
            r"$\omega_z$": h5file[f"fields/{time}/vorticity_z"][...],
        }
    
    return coords, vals

In [ ]:
def plot_single_snapshot(workdir, casename, time, petibmdir):
    """Plot a field of all cases at a single time.
    """

    # colored contour levels
    lvl1 = {
        r"$u$": numpy.linspace(-0.3, 1.2, 16),
        r"$v$": numpy.linspace(-0.8, 0.8, 17),
        r"$p$": numpy.linspace(-1.0, 0.5, 16),
        r"$\omega_z$": numpy.linspace(-5.0, 5.0, 11),
    }

    # contour line levels
    lvl2 = {
        r"$u$": numpy.linspace(0.0, 1.0, 6),
        r"$v$": numpy.linspace(-0.5, 0.5, 6),
        r"$p$": numpy.linspace(-0.75, 0.25, 6),
        r"$\omega_z$": numpy.linspace(-5.0, 5.0, 6),
    }

    # subplot locations
    locs = [(0, 0), (0, 1), (1, 0), (1, 1)]

    # data
    if casename == "PetIBM":
        coords, vals = read_petibm_snapshot(petibmdir, time)
    else:
        coords, vals = read_pinn_snapshot(workdir, casename, time)

    # figure
    fig, axs = pyplot.subplots(2, 2, sharex=True, sharey=True, figsize=(8, 6))

    if time == "steady":
        fig.suptitle(rf"Cylinder flow, $Re=200$, steady-state")
    else:
        fig.suptitle(rf"Cylinder flow, $Re=200$, $t={time}$")

    for i, (field, val) in enumerate(vals.items()):
        ax = axs[locs[i]]
        ct1 = ax.contourf(*coords[field], val, lvl1[field], cmap="cividis", extend="both")
        ct2 = ax.contour(*coords[field], val, lvl2[field], colors='black', linewidths=0.5)
        ax.clabel(ct2, lvl2[field], fmt="%1.1f", inline_spacing=0.25, fontsize="small")
        ax.add_artist(pyplot.Circle((0., 0.), 0.5, color="w", zorder=10))

        ax.set_title(field)
        ax.set_xlim(-3, 4)
        ax.set_ylim(-2.5, 2.5)
        ax.set_aspect("equal", "box")

        fmt1 = ticker.ScalarFormatter(useOffset=True, useMathText=True)
        fmt1.set_powerlimits((0, 0))
        cbar = fig.colorbar(ct1, ax=ax, ticks=lvl2[field], format=fmt1)
        cbar.ax.get_yaxis().set_offset_position("left")

    axs[1, 0].set_xlabel("x")
    axs[1, 1].set_xlabel("x")
    axs[0, 0].set_ylabel("y")
    axs[1, 0].set_ylabel("y")

    # save
    workdir.joinpath("figures", casename).mkdir(exist_ok=True)
    if time == "steady":
        fig.savefig(workdir.joinpath("figures", casename, f"contour.png"))
    else:
        fig.savefig(workdir.joinpath("figures", casename, f"contour-t{time}.png"))
    # pyplot.close(fig)

In [ ]:
# a widget to adjust which case at what time to show
slider = ipywidgets.FloatSlider(
    value=0.0, min=10.0, max=200.0, step=10.0,
    description="Time:", orientation="horizontal",
)

dropdown = ipywidgets.Dropdown(
    options=[
        "PetIBM",
        "nl6-nn512-npts6400-steady-raw",
        "nl6-nn512-npts6400-steady-swa",
        "nl6-nn512-npts6400-unsteady-raw",
        "nl6-nn512-npts6400-unsteady-swa",
        "nl6-nn512-npts6400-no-pinn-raw",
        "nl6-nn512-npts6400-no-pinn-swa",
        "nl8-nn512-npts6400-no-pin-raw",
        "nl8-nn512-npts6400-no-pin-swa",
    ],
    value="PetIBM", description="Case:"
)

out = ipywidgets.interactive_output(
    plot_single_snapshot,
    {
        "workdir": ipywidgets.fixed(workdir), "casename": dropdown,
        "time": slider, "petibmdir": ipywidgets.fixed(petibmdir)}
)

ipywidgets.VBox([out, slider, dropdown])

In [ ]:
def plot_force_coefficients(workdir, cases):
    """Plot a field of all cases at a single time.
    """
    copts = cycler("color", pyplot.cm.tab10.colors)()

    fig, ax = pyplot.subplots(1, 1, figsize=(8, 6))
    fig.suptitle(r"Lift ($C_L$) and drag ($C_D$) coefficients, $Re=200$")

    # line and label holders
    lines = []
    labels = []

    # add PetIBM results
    petibm = numpy.loadtxt(petibmdir.joinpath("output", "forces-0.txt"), dtype=float)

    lines.append((
        ax.plot(petibm[:, 0], petibm[:, 1]*2, "k-", lw=1, alpha=0.8)[0],
        ax.plot(petibm[:, 0], petibm[:, 2]*2, "k--", lw=1, alpha=0.8)[0]
    ))
    labels.append("PetIBM")

    for case in cases:
        with h5open(workdir.joinpath("outputs", case).with_suffix(".h5"), "r") as h5file:
            if h5file.attrs["unsteady"]:
                cd = h5file["coeffs/cd"][...]
                cl = h5file["coeffs/cl"][...]
                times = h5file["coeffs/times"][...]
            else:
                cd = numpy.full_like(petibm[:, 0], float(h5file["coeffs/cd"][...]))
                cl = numpy.full_like(petibm[:, 0], float(h5file["coeffs/cl"][...]))
                times = petibm[:, 0]
        
        c = next(copts)
        lines.append((
            ax.plot(times, cd, lw=1, ls="-", alpha=0.8, **c)[0],
            ax.plot(times, cl, lw=1, ls="--", alpha=0.8, **c)[0]
        ))
        labels.append(case)
    
    ax.set_xlim(0., 200.)
    ax.set_xlabel(r"$t$")

    ax.set_ylim(-0.8, 2.5)
    ax.set_ylabel("$C_D$ and $C_L$")

    ax.legend(
        lines, labels, handler_map={tuple: HandlerTuple(ndivide=None)},
        ncol=3, loc="upper center", bbox_to_anchor=(0.5, -0.15)
    )

    # save
    workdir.joinpath("figures").mkdir(exist_ok=True)
    fig.savefig(workdir.joinpath("figures", "drag-lift-coeffs.png"))
    # pyplot.close(fig)

In [ ]:
choices = ipywidgets.SelectMultiple(
    options=[
        "nl6-nn512-npts6400-steady-raw",
        "nl6-nn512-npts6400-steady-swa",
        "nl6-nn512-npts6400-unsteady-raw",
        "nl6-nn512-npts6400-unsteady-swa",
        "nl6-nn512-npts6400-no-pinn-raw",
        "nl6-nn512-npts6400-no-pinn-swa",
        "nl8-nn512-npts6400-no-pin-raw",
        "nl8-nn512-npts6400-no-pin-swa",
    ],
    value=[
        "nl6-nn512-npts6400-steady-raw",
        "nl6-nn512-npts6400-steady-swa",
        "nl6-nn512-npts6400-unsteady-raw",
        "nl6-nn512-npts6400-unsteady-swa",
        "nl6-nn512-npts6400-no-pinn-raw",
        "nl6-nn512-npts6400-no-pinn-swa",
        "nl8-nn512-npts6400-no-pin-raw",
        "nl8-nn512-npts6400-no-pin-swa",
    ],
    rows=10,
    description="Cases",
)

out = ipywidgets.interactive_output(
    plot_force_coefficients,
    {"workdir": ipywidgets.fixed(workdir), "cases": choices}
)

display(ipywidgets.HBox([out, choices]))

In [ ]:
def plot_training_history(workdir, casename, ws):
    """Plot figures related to training loss.
    """
    # fixed cycling kwargs
    kwargs = \
        cycler("color", pyplot.cm.tab10.colors[:3]) + \
        cycler("label", ["Raw data", "Moving averaged", "Moving minimum"]) + \
        cycler("linewidth", [0.3, 1.5, 1.5])
    kwargs = kwargs()

    kwr = next(kwargs)
    kwa = next(kwargs)
    kwm = next(kwargs)

    excludes = None
    data = read_tensorboard_data(workdir.joinpath(casename), excludes)

    fig, axs = pyplot.subplots(2, 1, sharex=False, sharey=False, figsize=(8, 6))
    fig.suptitle(rf"2D Cylinder flow, $Re=200$")
    
    # against steps
    axs[0].set_title("Aggregated loss v.s. iterations")
    axs[0].semilogy(data.index, data.loss, alpha=0.3, **kwr)
    axs[0].semilogy(data.index, data.loss.rolling(window=ws).mean(), **kwa)
    axs[0].semilogy(data.index, data.loss.rolling(window=ws).min(), **kwm)
    axs[0].axvline(200000, lw=0.3, ls="--", c="k")
    axs[0].set_xlabel("Iteration")
    axs[0].set_ylabel("Aggregated loss")
    axs[0].legend(loc=0)
    
    # against runtime
    axs[1].set_title("Aggregated loss v.s. run time")
    if "run_time" in data.columns:
        run_time = data.run_time/3600
        axs[1].semilogy(run_time, data.loss, alpha=0.3, **kwr)
        axs[1].semilogy(run_time, data.loss.rolling(window=ws).mean(), **kwa)
        axs[1].semilogy(run_time, data.loss.rolling(window=ws).min(), **kwm)
        try:
            axs[1].axvline(run_time.loc[200000], lw=0.3, ls="--", c="k")
        except KeyError:
            pass
    axs[1].set_xlabel("Run time (hours)")
    axs[1].set_ylabel("Aggregated loss")
    axs[1].legend(loc=0)

    # save
    workdir.joinpath("figures", casename).mkdir(exist_ok=True)
    fig.savefig(workdir.joinpath("figures", casename, f"loss-hist.png"))
    # pyplot.close(fig)

In [ ]:
# a widget to adjust which case at what time to show
slider = ipywidgets.IntSlider(
    value=10, min=1, max=200, step=2,
    description="Window size:", orientation="horizontal",
)

dropdown = ipywidgets.Dropdown(
    options=[
        "nl6-nn512-npts6400-steady",
        "nl6-nn512-npts6400-unsteady",
        "nl6-nn512-npts6400-unsteady-petibm",
        "nl6-nn512-npts25600-no-pinn",
        "nl6-nn512-npts25600-no-pinn-tfexplr",
        "nl6-nn512-npts50000-steady",
        "nl6-nn512-npts50000-unsteady",
        "nl6-nn768-npts25600-no-pinn",
        "nl8-nn512-npts6400-no-pinn",
        "nl8-nn512-npts50000-no-pinn",
    ],
    value="nl6-nn512-npts6400-steady", description="Case:"
)

out = ipywidgets.interactive_output(
    plot_training_history,
    {"workdir": ipywidgets.fixed(workdir), "casename": dropdown, "ws": slider}
)

display(ipywidgets.VBox([out, slider, dropdown]))

In [ ]:
def lr_simulator(**kwargs):
    """Plot learning rates.
    """

    steps = numpy.arange(0, kwargs["steps"], 100)
    cyclic = cyclic_exp_range(
        steps, kwargs["cyclic_gamma"], kwargs["cyclic_maxlr"],
        kwargs["cyclic_minlr"], kwargs["cyclic_halfcycle"]
    )
    explr = exponential(steps, kwargs["exp_gamma"], kwargs["exp_maxlr"])
    tfexplr = tf_exponential(
        steps, kwargs["tfexp_decay_rate"], kwargs["tfexp_decay_step"],
        kwargs["tfexp_maxlr"]
    )
    pyplot.semilogy(steps, cyclic, label="CyclicLR")
    pyplot.semilogy(steps, explr, label="ExponentialLR")
    pyplot.semilogy(steps, tfexplr, label="TFExponentialLR")
    pyplot.xlabel("Iteration")
    pyplot.xlabel("Learning rate")
    pyplot.legend(loc=0)

In [ ]:
# learning simulator
# ==================
# cyclic exp-range
cyclic = {
    "gamma": ipywidgets.FloatText(value=0.9999915, description="gamma:"),
    "maxlr": ipywidgets.FloatText(value=1e-2, description="max lr:"),
    "minlr": ipywidgets.FloatText(value=1e-6, description="min lr:"),
    "halfcycle": ipywidgets.IntText(value=5000, description="half cycle:"),
}

cyclic["panel"] = ipywidgets.VBox([
    ipywidgets.Label("Cyclic exp-range"),
    cyclic["gamma"],
    cyclic["maxlr"],
    cyclic["minlr"],
    cyclic["halfcycle"],
])

# exponential
explr = {
    "gamma": ipywidgets.FloatText(value=0.9999915, description="gamma"),
    "maxlr": ipywidgets.FloatText(value=1e-2, description="init lr"),
}

explr["panel"] = ipywidgets.VBox([
    ipywidgets.Label("Exponential"),
    explr["gamma"],
    explr["maxlr"],
])

# exponential
tfexplr = {
    "decay_rate": ipywidgets.FloatText(value=0.96, description="decay rate"),
    "decay_step": ipywidgets.IntText(value=5000, description="decay step"),
    "maxlr": ipywidgets.FloatText(value=1e-2, description="max lr"),
}

tfexplr["panel"] = ipywidgets.VBox([
    ipywidgets.Label("TF Exponential"),
    tfexplr["decay_rate"],
    tfexplr["decay_step"],
    tfexplr["maxlr"],
])

# total steps
steps = ipywidgets.IntText(value=1000000, description="Steps")

out = ipywidgets.interactive_output(
    lr_simulator,
    {
        "cyclic_gamma": cyclic["gamma"],
        "cyclic_maxlr": cyclic["maxlr"],
        "cyclic_minlr": cyclic["minlr"],
        "cyclic_halfcycle": cyclic["halfcycle"],
        "exp_gamma": explr["gamma"],
        "exp_maxlr": explr["maxlr"],
        "tfexp_decay_rate": tfexplr["decay_rate"],
        "tfexp_decay_step": tfexplr["decay_step"],
        "tfexp_maxlr": tfexplr["maxlr"],
        "steps": steps,
    }
)

display(
    ipywidgets.HBox([
        ipywidgets.VBox([out, steps]),
        ipywidgets.VBox([cyclic["panel"], explr["panel"], tfexplr["panel"]]),
    ])
)